In [9]:
from IPython.display import HTML, display
html = "<table><tr><td>Hanky Panky: alexandra lace robe<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/9/1/91r111_black_1.jpg' style='width: 900px;'></td><td>Macy's Calvin Klein<img src='https://cdna.lystit.com/200/250/tr/photos/saksfifthavenue/0400093782409-black-6d6c9f31-.jpeg' style='width: 900px;'></td></tr></table>"
display(HTML(html))

Hanky Panky: alexandra lace robe,Macy's Calvin Klein


# Cargamos los  datos 

In [10]:
import warnings
warnings.filterwarnings("ignore")

import pandas as pd
import seaborn as sns
import numpy as np
# to normalize the strings
import unicodedata
import matplotlib.pyplot as plt
%matplotlib inline

pd.set_option('display.max_colwidth', -1)
pd.set_option('display.max_rows', 1000)

pals = sns.hls_palette(8, l=.3, s=.8)
purple = sns.cubehelix_palette(11)
light_purple = sns.light_palette("purple",n_colors=8)
pink = sns.color_palette("husl", 8)[7]
sns.set_palette(pals)
fig_size=[12,9]
sns.set(font_scale = 1.4)
sns.set_style('white', {'legend.frameon':True})
#sns.set_style("white")
plt.rcParams["figure.figsize"] = fig_size

ModuleNotFoundError: No module named 'pandas'

In [8]:
brands_info = pd.read_csv('../input/cleaned-lingerie-data-from-different-brands/cleaned_data.csv')

NameError: name 'pd' is not defined


**analisis de las calificaciones de los productos**


In [ ]:
brands_name = ['Calvin Klein','Hanky Panky',"Victoria's Secret", 'Topshop', 'Wacoal', 'Amazon Calvin Klein',
               'Amazon Hanky Panky', "Amazon Victoria's Secret", 'Amazon Wacoal',
               'Macys Calvin Klein', 'Macys Hanky Panky', 'Macys Wacoal']

df = brands_info.groupby('brand').product_name.agg(['count', 'nunique'])
df = df.loc[brands_name]
df.rename(columns={'count':'Number of products (colors)', 'nunique': 'Number of unique models' })

Sabemos que hay 2 productos diferentes con el mismo nombre y color pero descripciones diferentes en Victoria's Secret, 2 en calvin Klein y 2 en Amazon Victoria's Secret. Debido a que es un número tan pequeño, no debería interferir con el análisis posterior. 

Respuestas: 

* Victoria's Secret ofrece la mayor cantidad de productos. Victoria's Secret de Amazon ofrece una cantidad bastante pequeña de productos. 

* La cantidad de productos del sitio web oficial de Calvin Klein es mayor que en Amazon y menor que en Macy's. 

* La cantidad de productos únicos en Amazon para todas las marcas es bastante pequeña, entre 36 y 1. 

* Macys de Calvin Klein y Wacoal tiene más producto que la OW. 

Sin embargo, solo hay 455 nombres de productos únicos de Victoria's Secret. Eso significa que esa gran cantidad de productos (3.5K) se debe a que la marca ofrece una gran variedad de colores diferentes. ¿No es eso lo que todas las mujeres quieren? 

 ¿Qué es en realidad ese producto de Victoria's Secret que vende Amazon?

In [ ]:
print(brands_info[brands_info['brand']=="Amazon Victoria's Secret"].product_name.iloc[1])

**¿Cómo difiere el precio según la marca?**


Es interesante distinguir los precios según la categoría de producto de cada marca para ver si todos los productos de una marca son caros o solo algunos. Calvin Klein y Hanky Panky solo tienen varias categorías, mientras que Victoria's Secreat es un poco desordenado. Intentaremos generalizar la categoría del producto mirando la URL del producto y la categoría del producto y buscando algunas palabras clave.

Varias definiciones que no conocía: <br>
“Un bralette es un sostén sin estructura. Por lo general, no tiene forro, ni acolchado, ni cables” (c) Cora Harrington <br>
"Un babydoll es un camisón o negligé corto, a veces sin mangas, holgado, destinado a..." (c) wiki.

Asumiremos que chemise y babydoll son el mismo tipo de categoría.

Hay una tendencia bastante interesante en los bralettes, por lo que la distinguiremos del sujetador.

Algunos de los sujetadores de Victoria's Secret tienen cuerpos en el nombre del producto.

Bras are considered to be all bras except for the sport bras. Sport bras are in the active wear category.

In [ ]:
def manipul_regex(str_array):
    string = "|".join(str_array)
    return "(^|\s)(" + string + ")(\s|$)"

def categorize(data):
    panties = manipul_regex(["thong","g string","pant","v kini","boypant","pants","panty","thongs","panties",
                             "ladypant","knickers","thong","twist knicker","brief","boyshort",
                             "lace v front short","signature lace bike short","side tie bikini",
                             "signature lace string bikini","tanga","panty","hipster","vikini","b.sleek bikini",
                             "b.sultry bikini","b.sumptuous bikini","b.tempt'd lace kiss bikini",
                             "cheekster", 'boypants','ladypants', 'boyshorts', 'hiphugger', 
                             'pink high leg logo bikini', 'pink shortie', 'pink logo bikini',
                             'lace sexy shortie', 'body base shorty', 'b.splendid bikini'
                            ])  
    
    bodys = manipul_regex(["bodysuit","teddy","wink plaything"])
    bras = manipul_regex(["bra","bustier","strapless","balconette","bandeau",
                          'pink palm lace strappy push up','body by victoria unlined demi', 
                          'pink daisy lace racerback push up', 
                          'pink wildflower lace high neck push up',
                          'pink wear everywhere super push', 'pink seamless lightly lined racerback',
                          'body by wacoal seamless underwire','basic beauty wireless contour',
                          'pink lace lightly lined triangle', 'lace cross front unlined halter'
                            ])
    
    activewear = manipul_regex(['sports bra', 'sport bra', 'sport bralette', 'sports bralette'])
    suspenders = manipul_regex(["suspenders","belt"])
    bralettes = manipul_regex(["bralettes","bralette", 'bralet'])
    tops = manipul_regex(["tops","top"])
    babydoll = manipul_regex(["babydoll","camisole and bikini set by bluebella","chemise"])
    shorts = manipul_regex(["short","shorts","chiffon tap pant"])
    socks = manipul_regex(["socks"])
    pack = manipul_regex(["pack"])
    maternity = manipul_regex(["maternity"])
    basque = manipul_regex(["basque"])
    slip = manipul_regex(["slip"])
    bralette = manipul_regex(["bralette","bralettes"])
    garter = manipul_regex(["garter"])
    robe = manipul_regex(["robe", "kimonos","kimono"])
    camisole = manipul_regex(["camisole","cami","tank"])
    shapers = manipul_regex(["bike short","shaper","bodybriefer"])
    eyemask = manipul_regex(["eyemask"])
    hoodie = manipul_regex(["hoodie"])
    sleepwear = manipul_regex(["pajama","sleep set","sleepwear","modern cotton lounge pants"])
    rompers = manipul_regex(["romper"]) 
    
    dict_cats = [['hoodie',hoodie],['eyemask', eyemask],[ 'shapers',shapers],
                 [ 'garter', garter],['bralette',bralette],
                 ['slip',slip],[ 'basque',basque],['maternity',maternity],
                 ['pack',pack],['socks',socks],['shorts',shorts],['robe', robe],
                 ['tops',tops],
                 ['suspenders',suspenders],['rompers', rompers],
                 ['bras',bras],['panties',panties], ['babydoll', babydoll],
                 ['bodys',bodys],['sleepwear', sleepwear],['bralettes', bralettes], ['activewear', activewear]
                ]
    #because there is a babydool with g-string and teddy with thong

    
    data.loc[(data['brand'].str.contains('Macys')), 'product_category_gen'] = 'Other'
    data.loc[(data['brand']== "Victoria's Secret"), 'product_category_gen'] = 'Other'
    data.loc[(data['brand']== "Hanky Panky"), 'product_category_gen'] = 'Other'
    
    dict_cats_pro = [['camisoles',camisole]]
    for items in dict_cats_pro:
        naming, reg = items
        data.loc[(data['product_name'].str.contains(reg, case=False)), 'product_category_gen'] = naming
        data.loc[(data['description'].str.contains(reg, case=False)), 'product_category_gen'] = naming

    for items in dict_cats:
        naming, reg = items
        data.loc[(data['product_name'].str.contains(reg, case=False)), 'product_category_gen'] = naming
    
    #data.loc[(data['description'].str.contains('bralette|bralettes', case=False))&(data['product_name'].str.contains('bra')), 'product_category_gen'] = 'bralettes'

    #data.loc[(data['brand']== "Calvin Klein")&(data['product_name'].str.contains('hoodie', case=False)), 'product_category_gen'] = 'hoodie'        
    data.loc[(data['brand'].str.contains("Macys"))&(data['product_name'].str.contains('bikini', case=False)), 'product_category_gen'] = 'panties'
    data.loc[(data['brand']== "Victoria's Secret")&(data['product_name'].str.contains('dream angels new! lace-up pleated slip', case=False)), 'product_category_gen'] = 'babydoll'
    data.loc[(data['brand']== "Victoria's Secret")&(data['product_name'].str.contains('very sexy lace plunge garter slip', case=False)), 'product_category_gen'] = 'babydoll'
    #data.loc[(data['brand']== "Victoria's Secret")&(data['pdp_url'].str.contains('swim', case=False)), 'product_category_gen'] = 'swimsuits'
    data.loc[(data['descr_conc'].str.contains('bralette|bralettes'))&(data['product_name'].str.contains('bandeau', case=False)), 'product_category_gen'] = 'bralettes'

    data.loc[(data['brand']== "Victoria's Secret")&(data['pdp_url'].str.contains('activewear', case=False)), 'product_category_gen'] = 'activewear'
    data.loc[(data['pdp_url'].str.contains('swim', case=False)), 'product_category_gen'] = 'swimsuits'
    data.loc[(data['descr_conc'].str.contains('swim', case=False)), 'product_category_gen'] = 'swimsuits'

    data.loc[(data['brand']== "Calvin Klein")&(data['pdp_url'].str.contains('activewear', case=False)), 'product_category_gen'] = 'activewear'

    data['product_category_gen'] = data['product_category_gen'].apply(lambda x: x.replace("women - lingerie & shapewear - ", ""))

    data.loc[(data['brand']== "Topshop")&(data['product_name'].str.contains('calvin klein', case=False)), 'product_category_gen'] = 'calvin klein'
    data.loc[(data['brand']== "Topshop")&(data['pdp_url'].str.contains('body', case=False)), 'product_category_gen'] = 'bodys'


    data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains(manipul_regex(['top|crop']), case=False)), 'product_category_gen'] = 'bras'
    data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('bikini', case=False)), 'product_category_gen'] = 'panties'
    data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('slip', case=False)), 'product_category_gen'] = 'slip'
    data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('t-shirt', case=False)), 'product_category_gen'] = 'camisoles'
    #data.loc[(data['brand']== "Hanky Panky")&(data['descr_conc'].str.contains('panty', case=False)), 'product_category_gen'] = 'panties'


    data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('low rise thong rose bouquet of|original rise thong rose bouquet of', case=False)), 'product_category_gen'] = 'pack'
    #data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('low rise thong rose bouquet of 12', case=False)), 'product_category_gen'] = 'pack'
    #data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('original rise thong rose bouquet of 6', case=False)), 'product_category_gen'] = 'pack'
    #data.loc[(data['brand']== "Hanky Panky")&(data['product_name'].str.contains('original rise thong rose bouquet of 12', case=False)), 'product_category_gen'] = 'pack'
    data.loc[(data['product_name'].str.contains(manipul_regex(['pack', 'kiss packaged']), case=False)), 'product_category_gen'] = 'pack'

    #data.loc[(data['product_category'].str.contains('panty sets', case=False)), 'product_category_gen'] = 'panty sets'
    #data.loc[(data['brand']== "Victoria's Secret")&(data['product_category'].str.contains('cami & short set', case=False)), 'product_category_gen'] = 'sleep shorts & top'

    #['widow and thong', 'sultry balconette bra, thong and garter belt']
    #bra_panties_set = ['widow & thong', 'bahama mama lace',
    #                   'bra &','b.sultry balconette bra, thong and garter belt',
    #                   '^(?=.*bra)(?=.*(bikini|panty|tanga|hipster|thong|short|trunk|high cut brief|knickers))']
    #data.loc[(data['product_category'].str.contains('lingerie sets', case=False)), 'product_category_gen'] = 'bra panties sets'
    data.loc[(data['descr_conc'].str.contains('widow and thong', case=False)), 'product_category_gen'] = 'bra panties sets'


    data.loc[(data['product_name'].str.contains('sleeve top', case=False)), 'product_category_gen'] = 'tops'
    data.loc[(data['product_name'].str.contains('sleep set', case=False)), 'product_category_gen'] = 'sleepwear'
    data.loc[(data['product_category'].str.contains('sleepwear'))&(data['pdp_url'].str.contains('sleepwear', case=False)), 'product_category_gen'] = 'sleepwear'
    data.loc[(data['brand']=="Victoria's Secret")&(data['product_name'].isin(['pink wear everywhere t shirt bra','pink lace t shirt bra'])), 'product_category_gen'] = 'bras'
    data.loc[(data['brand']=="Macys Wacoal")&(data['product_name'].str.contains('embrace lace soft cup wireless bra')), 'product_category_gen'] = 'bras'
    data.loc[(data['brand']=="Victoria's Secret")&(data['product_name']=='pink super soft beach cheeky cropped tank'), 'product_category_gen'] = 'camisoles'
    # we will assume that there is no such thing as sport underwear
    data.loc[(data['product_name'].str.contains(bras))&(data['brand']== "Calvin Klein")&(data['pdp_url'].str.contains('sports-bras', case=False)), 'product_category_gen'] = 'activewear'
    data.loc[(data['product_name'].str.contains(bralettes))&(data['brand']== "Calvin Klein")&(data['pdp_url'].str.contains('sports-bras', case=False)), 'product_category_gen'] = 'activewear'
    data.loc[(data['brand']=='Topshop')&(data['product_name'].isin(['topshop branded sporty bralet','seamless sporty branded bra','seamless sporty bra', 'sporty bra','sporty branded bra'])), 'product_category_gen'] = 'activewear'
    data.loc[(data['brand']=='Macys Wacoal')&(data['product_name'].str.contains('sport high impact underwire bra')), 'product_category_gen'] = 'activewear'
    data.loc[(data['brand']=="Victoria's Secret")&(data['product_name'].isin(['body by victoria daisy lace sleep tee','macrame lace mix sleep short','double v sleep romper'])), 'product_category_gen'] = 'sleepwear'
    data.loc[(data['brand']=="Hanky Panky")&(data['product_name'].isin(['violet spray chiffon sleep top'])), 'product_category_gen'] = 'sleepwear'

    data.loc[(data['pdp_url']=='https://www.amazon.com/-/dp/b06wrtg5d8?th=1&psc=1'), 'product_category_gen'] = 'bra panties sets'
    data.loc[(data['product_name'].str.contains('maternity')), 'product_category_gen'] = 'maternity'

    return data

In [ ]:
brands_info = categorize(brands_info)

In [ ]:
brands_info['product_category_gen'].unique()

In [ ]:
display(brands_info[(brands_info['product_category_gen']=='lingerie')])

Curiosamente, Victoria's Secret no tiene mucha ropa para dormir. Parece que los enlaces eran diferentes cuando se eliminaron los datos o algo andaba mal con el eliminador.

In [ ]:
display(brands_info[(brands_info['pdp_url'].str.contains('sleep')) & (brands_info['brand']=="Victoria's Secret")].product_name.unique())

**modelador de tema en la  descrpcion del producto**

**¿Qué productos ofrecen las marcas?**

La pregunta es si consideras que un producto es único por un modelo y color o solo por un modelo. Considere la siguiente situación: La tienda vende 2 modelos de sujetadores con 25 colores cada uno y 10 modelos de bragas con 3 colores cada uno. Si un producto se define por un modelo y un color, dirías que el 62,5% de los productos son sujetadores. Si define un producto solo por un modelo, entonces el 83% de los productos son bragas.

**Amazonas**:
* Amazon Calvin Klein tiene muchos paquetes (paquetes de 3, 4 o 5 bragas). (Tal vez esa es la razón por la que Clavin Klein en Amazon es más caro que en el sitio web oficial).
* Amazon Hanky ​​Panky ofrece solo (3 tipos de) bragas en una amplia variedad de colores.
* Como sabemos, Amazon Victoria's Secret ofrece solo un tipo de ropa deportiva (y revelaré que es un sostén) en diferentes colores.
* Wacoal ofrece los mismos modelos de sujetadores (3 modelos), bralettes (1 modelo) y bragas (2 modelos) en muchos colores diferentes.

**Sitios web oficiales**:
* Calvin Klein ofrece muchos colores diferentes de bragas. El producto principal (39%) son las bragas (incluidas las bragas 3for33) y el siguiente son los sujetadores (25%). Además, ofrecen bastante ropa de dormir.
* La mitad de los productos de Hanky ​​Panties son bragas. Y hay muchos más bralettes (38 artículos) que sujetadores (9 artículos). Hay muchos colores de bragas también.
* Los principales productos de Victoria's Secret son las bragas y la ropa deportiva (la mayoría de los cuales son sujetadores) (también en muchos colores), seguidos de sujetadores y bralettes.
* El principal producto de Wacoal son las bragas. Sujetadores y bragas comprende el 73% de los productos.
* Los principales productos de Topshop son sujetadores (30%) y bragas (48%).

**Macys**:
* Macys Calvin Klein vende más bragas y sujetadores que el sitio web oficial. En general, Macys CK vende más productos y algunas otras categorías (pero en una pequeña cantidad de productos) que el sitio web oficial. Sin embargo, la tendencia es la misma: muchos colores de las bragas.
* El sitio web oficial de Hanky ​​Panky (OW) vende muchos más productos que Macys y Macys HP no vende sujetadores. Pero Macys tiene más colores de bralettes.
* Macys Wacoal vende más productos que OW pero OW tiene más bralettes de diferentes colores para ofrecer.


**Adicional**:
* Casi todas las marcas venden tirantes en pequeñas cantidades (excepto Calvin Klein y Amazon).
* Los productos de maternidad (inkl. sujetadores y bragas) son vendidos por Topshop (14 productos) y productos de forma por Macys Wacoal (7 productos)


* Topshop tiene una categoría Calvin Klein que no se especifica como tal en la marca.

In [ ]:
for name, group in brands_info.groupby('brand'):
    print(name)
    
    df = group.drop_duplicates(['product_name'],inplace=False)
    df = pd.DataFrame(df.groupby('product_category_gen').size()).sort_values(by = 0, ascending=False).transpose()
    df_per = df.apply(lambda r: r/r.sum() * 100, axis=1)
    df = df_per.append(df)
    df.index = ['percent (m)', 'absolut (m)']
    
    
    tab = pd.DataFrame(group.groupby('product_category_gen').size()).sort_values(by = 0, ascending=False).transpose()
    cols = tab.columns.tolist()
    tab_per = tab.apply(lambda r: r/r.sum() * 100, axis=1)
    tab = tab_per.append(tab)
    tab.index = ['percent (m+c)', 'absolut (m+c)']
    
    display(tab.append(df)[cols])

In [ ]:
ind = brands_info[brands_info['product_category_gen']=='calvin klein'].index
brands_info.drop(ind, inplace = True)

**¿Cuál es el promedio de colores por producto?**

Entonces, ahora sabemos que Amazon Victoria's Secret tiene 9 colores diferentes de ese sujetador deportivo.
* Victoria's Secret ofrece la mayor cantidad de colores diferentes por producto. Además, hay algunos valores atípicos muy interesantes. ¡Hay varios productos que tienen más de 40 colores!
* Topshop tiene muy pocos colores para elegir. Puede verse influenciado por los valores perdidos.

In [ ]:
colors_df = pd.DataFrame()    
for name, group in brands_info.groupby('brand'):
    colors = group.groupby(['product_name']).color.nunique()
    colors_df = colors_df.append(pd.DataFrame({'color_num': colors, 'brand': name}))

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(20,8))
ax = sns.boxplot(x='color_num', y='brand', data=colors_df, ax=axes, medianprops={"zorder":3})
ax = sns.swarmplot(x="color_num", y="brand", data=colors_df, color='grey', alpha=0.7)
ax.xaxis.grid(True)
ax.set_xlabel('Number of colors per product', fontsize=17);
ax.tick_params(axis='both', which='major', labelsize=17);


In [ ]:
colors_df.groupby('brand').agg(['median', 'mean', 'count']).transpose()

Echemos un vistazo más de cerca a los productos que tienen más de 35 colores.
(Amazon Hanky Panky tiene 6 colores como promedio, pero solo hay 3 productos únicos, dos con 10 y 9 colores y uno con 1 color).

Algunos modelos de sujetadores y braguitas tienen una extraordinaria cantidad de colores :). 'the player racerback sport bra' tiene la mayor cantidad de colores, seguido de 'seamless hiphugger panty' de Victoria's Secret.

In [ ]:
display(colors_df[colors_df['color_num']>=35])
del colors_df

In [ ]:
#brands_info[brands_info['product_name'] == 'victoria sport the player racerback sport bra by victoria sport'].color.unique()

**Pregunta**: ¿Qué color domina?

Entonces, parece que la mayoría de los productos tienen color negro (lo que también se espera en realidad) y la segunda mayoría de color es en su mayoría blanco o desnudo, excepto en algunas marcas. Y Macys Hanky Panky es rojo ;).

Me pregunto de qué color es el 'áster salvaje' que ofrece Wacoal.

In [ ]:
colors_first = brands_info.groupby(['brand','color'])['color'].agg({'count_fir':'count'})
df_count_first = colors_first.groupby(level=0).agg('idxmax')
df_count_first = colors_first.loc[df_count_first['count_fir']].reset_index()

data_mod = brands_info[~brands_info['color'].isin(df_count_first.color.unique())]
colors_second = data_mod.groupby(['brand','color'])['color'].agg({'count_sec':'count'})
df_count_second = colors_second.groupby(level=0).agg('idxmax')
df_count_second = colors_second.loc[df_count_second['count_sec']].reset_index()

df = df_count_first.merge(df_count_second, on='brand')
df.columns = ['brand', '1st color', 'count 1st color', '2nd color', 'count 2nd color']
display(df)

for x in [df_count_first, df_count_second, df, data_mod, colors_first]:
    del x

Wild auster color:
<img src='https://images-na.ssl-images-amazon.com/images/I/91Cai0vSt-L._SL1500_.jpg' style='width: 200px;'>
    


In [ ]:
main_categories = ['bralettes','activewear','bras','panties','bodys','pack','babydoll', 'swimsuits', 'sleepwear']
brands_info['product_category'] = brands_info['product_category_gen']
brands_info['category'] = brands_info['product_category_gen']
brands_info.loc[~brands_info['category'].isin(main_categories), 'category'] = 'Other'
del brands_info['product_category_gen']

## 2.1 MRP - Precio minorista máximo

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(20,5))
ax = sns.boxplot(x='mrp', y= 'brand', data = brands_info, palette = purple, order = brands_name) 

**Pregunta 11**: Hanky Panky tiene realmente varios valores atípicos. ¡Y algunos cuestan más de 600 USD! ¿Cuáles son esos productos?

In [ ]:
brands_info[(brands_info['mrp']>300)&(brands_info['brand']=='Hanky Panky')].product_name.unique()

¿Quieres un paquete de 30 tangas de diferentes colores por 600 USD? ;)

## Distribución MRP

In [ ]:
def general_plot(money = 'mrp'):
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(20,5))
    ax = sns.boxplot(x=money, y= 'brand', data = brands_info[brands_info[money]<150], palette = purple, order = brands_name) 
    _ = ax.set_xticklabels(ax.get_xticklabels(), fontsize=13)


    fig, axes = plt.subplots(nrows=1, ncols=2,figsize=(20,4))
    ax.xaxis.grid(True)
    ax = sns.barplot(y=money, x = 'brand', data = brands_info, palette = light_purple[1:], ax = axes[0], order = brands_name)
    ax.yaxis.grid(True)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=13)
    ax = sns.barplot(y=money, x = 'brand', data = brands_info, palette = light_purple[1:], estimator=np.median, ax = axes[1], order = brands_name)
    ax.yaxis.grid(True)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90, fontsize=13)

* Macy's Calvin Klein, Macy's Hanky Panky y Hanky Panky tienen algún producto con mrp > 120. Necesita un vistazo.
* Parece que Calvin Klein es un poco más caro que Victoria's Secret.
* Wacoal parece ser la marca más barata. Pero Macy's Wacoal es mucho más caro que OW.
* Amazon Calvin Klein es más caro que OW Calvin Klein. (También Macy's es un pero más caro)

In [ ]:
general_plot()

## Distribución de precios
* Parece que Calvin Klein es un poco más caro que Victoria's Secret.
* Wacoal y Victoria's Secret parecen ser las marcas más baratas. Pero Macy's Wacoal es mucho más caro que OW.
* Amazon rebajado Calvin Klein y Calvin Klein tienen los mismos precios.

In [ ]:
general_plot(money = 'price')

**Pregunta 11**: ¿Qué son todos esos productos con mrp > 120?

Esas categorías son definitivamente las más caras :)

In [ ]:
brands_info[(brands_info['mrp']>120) & (brands_info['category']!='pack')].product_category.unique()

Se ve bien, ¿no?

In [ ]:
brands_info[brands_info['product_name']=='alexandra lace robe']

In [ ]:
from IPython.display import HTML, display
html = "<table><tr><td>Hanky Panky: sophia lace bodysuit<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/7/9/798504_black_4.jpg' style='width: 800px;'></td><td>Hanky Panky: short european flannel robe<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/2/v/2vr114_1.jpg' style='width: 800px;'></td><td>Hanky Panky: alexandra lace robe<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/9/1/91r111_black_1.jpg' style='width: 900px;'></td><td>Hanky Panky: wink plaything<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/4/t/4t8504_black_1.jpg' style='width: 900px;'></td><td>Hanky Panky: victoria chemise with g string<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/9/4/945901_2.jpg' style='width: 900px;'></td><td>Hanky Panky: heather jersey kimono robe<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/6/8/68r222_wine_1.jpg' style='width: 900px;'></td><td>Macy's Calvin Klein<img src='https://cdna.lystit.com/200/250/tr/photos/saksfifthavenue/0400093782409-black-6d6c9f31-.jpeg' style='width: 900px;'></td></tr></table>"
display(HTML(html))

**Pregunta 11**: ¿Cuál es el artículo más caro (por mrp) en cada marca?

In [ ]:
df = pd.DataFrame(brands_info.groupby('brand').mrp.max()).reset_index()
df2 = pd.DataFrame(brands_info.groupby('brand').price.max()).reset_index()
df = df.merge(df2, on='brand')
display(df)

In [ ]:
expensive_items_mrp = pd.DataFrame()
expensive_items_price = pd.DataFrame()

for ind, row in df.iterrows():
    df_temp = brands_info[(brands_info['brand']==row['brand'])&(brands_info['mrp']==row['mrp'])]
    names = df_temp.product_name.unique()
    expensive_items_mrp = expensive_items_mrp.append(df_temp.iloc[0])

    df_temp = brands_info[(brands_info['brand']==row['brand'])&(brands_info['price']==row['price'])]
    listings = list(set(df_temp.product_name) - set(names))
    if len(listings)!=0:
        df_temp = df_temp[(df_temp['product_name'].isin(listings))]
        expensive_items_price = expensive_items_price.append(df_temp.iloc[0])
#display(expensive_items_mrp.head(5))

Entonces, el artículo más caro en nuestro conjunto de datos es de Hanky Panky y son 30 pares de tangas.

In [ ]:

html = "<table><tr><td>Calvin Klein<img src='https://cdnd.lystit.com/1040/1300/n/photos/shopbop/CKLEN4233210437-Black-04ad40dc-.jpeg' style='width: 900px;'></td><td>Hanky Panky<img src='http://www.hankypankyltd.netdna-cdn.com/media/catalog/product/cache/1/image/db978388cfd007780066eaab38556cef/4/8/48pk30_emul_1_1.jpg' style='width: 800px;'></td><td>Victorias Secret<img src='https://dm.victoriassecret.com/p/760x1013/tif/b4/4f/b44f5a12e28c4b8c9dcab27faae01013/V493406_OF_F.jpg' style='width: 900px;'></td><td>Topshop<img src='http://media.topshop.com/wcsstore/TopShopDE/images/catalog/TS43E04LLAC_Zoom_F_1.jpg' style='width: 900px;'></td><td>Wacoal<img src='https://a248.e.akamai.net/f/248/9086/10h/origin-d5.scene7.com/is/image/WacoalAmerica/953237_004?qlt=85,1&op_sharpen=1&resMode=bilin&op_usm=1,1,9,0&rect=0,0,261,344&scl=5.816860465116279&id=VFprq1' style='width: 900px;'></td></tr></table>"
display(HTML(html))
html = "<table><tr><td>Amazon Calvin Klein<img src='https://images-na.ssl-images-amazon.com/images/I/61wmAanrDzL._UL1500_.jpg' style='width: 900px;'></td><td>Amazon Victorias Secret<img src='https://images-na.ssl-images-amazon.com/images/I/71CxBauOwPL._UL1500_.jpg' style='width: 900px;'></td><td>Amazon Hanky Panky<img src='https://images-na.ssl-images-amazon.com/images/I/71gd1OukSaL._UL1500_.jpg' style='width: 900px;'></td><td>Amazon Wacoal<img src='https://images-na.ssl-images-amazon.com/images/I/81uAK8Lvx8L._UL1500_.jpg' style='width: 900px;'></td><td>Macys Calvin Klein<img src='https://cdna.lystit.com/200/250/tr/photos/saksfifthavenue/0400093782409-black-6d6c9f31-.jpeg' style='width: 900px;'></td><td>Macys Hanky Panky<img src='https://image.shopittome.com/apparel_images/fb/hanky-panky-hanky-panky-eyelash-lace-bodysuit-4t8504-abv6af8ff20_zoom.jpg' style='width: 900px;'></td><td>Macys Wacoal<img src='https://slimages.macysassets.com/is/image/MCY/products/0/optimized/3831480_fpx.tif?op_sharpen=1&wid=400&hei=489&fit=fit,1&$filterlrg$' style='width: 900px;'></td></tr></table>"
display(HTML(html))

**Question 12**: What is the most expensive discounted item (by price) in each brand?

In [ ]:
#display(expensive_items_price)

In [ ]:
html = "<table><tr><td>Amazon Calvin Klein<img src='https://images-na.ssl-images-amazon.com/images/I/81LEl1kn5oL._UL1500_.jpg' style='width: 200px;'></td><td>Macys Wacoal<img src='https://slimages.macysassets.com/is/image/MCY/products/5/optimized/3640215_fpx.tif??op_sharpen=1&fit=fit,1&$filterlrg$&wid=1200&hei=1467' style='width: 200px;'></td><td>Wacoal<img src='https://images.prod.meredith.com/product/96a181a21f45ac2c2183d2076f789d9d/53fc48ce1c2d384f512eaa489f2011980c2dc667471d96e0ad01570626b111b8/l/womens-b-temptd-by-wacoal-b-sultry-chemise' style='width: 200px;'></td></tr></table>"
display(HTML(html))

So, back to the reality.

## 2.1.1 ¿Qué son las distribuciones MRP?

In [ ]:
def create_plots(cols, swarm = False, inner = None, money = 'mrp'):
    df = brands_info[brands_info[money]<=100]
    df2 = df[df['brand'].isin(cols)]
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(15,5))
    ax = sns.violinplot(y=money, x='brand', data=df2, split=True,inner=inner, ax = axes, palette = light_purple)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90,fontsize=13)
    if swarm:
        ax = sns.swarmplot(x="brand", y=money, data=df2, color="black", alpha=.5, ax = axes)
    ax.yaxis.grid(True)
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(20,6))
    ax = sns.boxplot(y=money,hue='category',x = 'brand',data = df2, palette=sns.color_palette("husl", 10))
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90,fontsize=13)
    ax.yaxis.grid(True)
    _ = ax.set_ylabel(money)    

**Pregunta 13**: Qué productos causan distribuciones y cuáles son las razones por las cuales las mismas marcas tienen diferentes rangos de mrp en diferentes sitios.

In [ ]:
create_plots(['Calvin Klein', "Victoria's Secret", 'Topshop', 'Wacoal', 'Hanky Panky', 'Amazon Calvin Klein', 'Macys Calvin Klein','Amazon Hanky Panky', 'Macys Hanky Panky','Amazon Wacoal', 'Macys Wacoal'])

### Sitios web oficiales

* La mayoría de los productos de Victoria's Secret tienen un precio de alrededor de 12 dólares. Esas son las bragas. Los siguientes productos principales tienen un precio de alrededor de 38 dólares. Y eso son los sujetadores y bodys y otros productos.<br>
* La mayoría de los productos de Calvin Klein tienen un precio de alrededor de 13 dólares. <br>
* La mayoría de los productos de Topshop rondan los 12 dólares y los productos de Wacoal rondan los 13 dólares.
* Hanky Panky parece ser la marca más cara

* Los picos alrededor de 13 son bragas y los picos hacia arriba son sujetadores, bralettes, etc.

In [ ]:
create_plots(['Calvin Klein', "Victoria's Secret", 'Topshop', 'Wacoal', 'Hanky Panky'])

### Calvin Klein

**Pregunta 14:** ¿Por qué Amazon Calvin Klein es más caro que el sitio web de Calvin Klein? (También Macy's es un poco más caro)?

* El sitio web oficial de Calvin Klein vende más bragas menos costosas (alrededor de 13 USD) que Amazon Calvin Klein (alrededor de 20 USD)
* Los paquetes caros (2, 3, 5 juegos) se venden principalmente en Amazon.
* Macy's vende bodys y babydolls caros.

* Como sabemos por la Pregunta 4, alrededor del 55 % de los productos de Calvin Klein son esas bragas económicas. Esa es la razón por la cual, en promedio, Calvin Klein es menos costoso que Amazon. Pero piensa en los paquetes que ofrece Amazon. El MRP para una braga puede ser menor en un paquete de 5, pero no podrá elegir los colores que desea si es muy exigente: P

In [ ]:
create_plots(['Calvin Klein', 'Amazon Calvin Klein', 'Macys Calvin Klein'], swarm = True)

In [ ]:
df = brands_info[(brands_info['brand'].isin(['Amazon Calvin Klein','Calvin Klein']))]
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(14,5))
ax = sns.violinplot(y="mrp", hue ='brand', x = 'product_category', split=True, data=df, ax = axes, palette = [light_purple[2], light_purple[5]])
ax.yaxis.grid(True)

So, in fact if you buy packs on Amazon you will pay less for each panty (around 11USD or discounted 8 USD) than on official website.

In [ ]:
def find_price_per_panty(brand):
    
    pack_panties = brands_info[(brands_info['brand']==brand)&(brands_info['category']=='pack')]
    pack_panties = pack_panties[~pack_panties['product_name'].str.contains(manipul_regex(['bra', 'bralette']))]
    def find_digit(x):
        for s in x.split():
            if s.isdigit():
                return int(s)
    pack_panties['panties_num'] = pack_panties['product_name'].apply(lambda x: find_digit(x))
    mrp = (pack_panties['mrp']/pack_panties['panties_num'])
    price = (pack_panties['price']/pack_panties['panties_num'])
    return [mrp,price]

In [ ]:
hankypanky_mrp, hankypanky_price = find_price_per_panty('Hanky Panky')
macysck_mrp, macysck_price = find_price_per_panty('Macys Calvin Klein')
amazonck_mrp, amazonck_price = find_price_per_panty('Amazon Calvin Klein')


In [ ]:
packs_price = pd.DataFrame({'brand': 'Amazon Calvin Klein', 'mrp': amazonck_mrp})
packs_price = packs_price.append(pd.DataFrame({'brand': 'Macys Calvin Klein', 'mrp': macysck_mrp})) 
packs_price = packs_price.append(pd.DataFrame({'brand': 'Hanky Panky', 'mrp': hankypanky_mrp})) 

In [ ]:
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10,2))
ax = sns.boxplot(y='brand', x='mrp',data = packs_price, ax = axes, medianprops={"zorder":3},
                palette = [light_purple[2],light_purple[6]])
ax = sns.swarmplot(y='brand', x='mrp',data = packs_price, color = 'grey',alpha=0.7)

In [ ]:
packs_price = pd.DataFrame({'brand': 'Amazon Calvin Klein', 'price': amazonck_price})
packs_price = packs_price.append(pd.DataFrame({'brand': 'Macys Calvin Klein', 'price': macysck_price})) 
packs_price = packs_price.append(pd.DataFrame({'brand': 'Hanky Panky', 'price': hankypanky_price})) 

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(8,2))
ax = sns.boxplot(y='brand', x='price',data = packs_price, ax = axes, medianprops={"zorder":3},
                 palette = [light_purple[2],light_purple[6]])
ax = sns.swarmplot(y='brand', x='price',data = packs_price, color = 'grey',alpha=0.7)
ax.set_xlabel('price per panty from a pack')

In [ ]:
packs_price = pd.DataFrame({'brand': 'Amazon Calvin Klein', 'mrp': amazonck_mrp})
packs_price = packs_price.append(pd.DataFrame({'brand': 'Macys Calvin Klein', 'mrp': macysck_mrp})) 

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(10,2))
ax = sns.boxplot(y='brand', x='mrp',data = packs_price, ax = axes, medianprops={"zorder":3},
                palette = [light_purple[2],light_purple[6]])
ax = sns.swarmplot(y='brand', x='mrp',data = packs_price, color = 'grey',alpha=0.7)
ax.set_xlabel('price per panty from a pack',fontsize=13)
_= ax.tick_params(axis='both', which='major', labelsize=13)

### Wacoal

* Amazon sells a bit more expensive panties of Wacoal than on the official website, whereas on Macys there is a nice gradient of all prices.
* The expensive products in Other category are mainly shapers that are not sold on other sites.

In [ ]:
create_plots(['Wacoal', 'Amazon Wacoal', 'Macys Wacoal'], swarm = True)

## Hanky ​​Panky
* El sitio web oficial de HP y Macy vende casi el mismo rango de precios de productos, mientras que Amazon busca bragas que también son un poco más caras.

In [ ]:
create_plots(['Hanky Panky', 'Amazon Hanky Panky', 'Macys Hanky Panky'],swarm = True)

In [ ]:
def create_plots(cols, swarm = False, inner = None, money = 'mrp'):
    df = brands_info[brands_info[money]<=100]
    df2 = df[df['brand'].isin(cols)]
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(15,5))
    ax = sns.violinplot(x=money, y='brand', data=df2, split=True,inner=inner, ax = axes, palette = [light_purple[0], light_purple[2], light_purple[5]])
    if swarm:
        ax = sns.swarmplot(x="brand", y=money, data=df2, color="black", alpha=.5, ax = axes)
    ax.yaxis.grid(True)
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(20,5))
    ax = sns.boxplot(y=money,hue='category',x = 'brand',data = df2, palette=sns.color_palette("husl", 9))
    ax.yaxis.grid(True)
    _ = ax.set_ylabel(money)

## El secreto de Victoria
Como sabemos, Amazon vende solo un tipo de sujetador de Victoria's Secret. Sin embargo, todavía tenemos una distribución de precios, lo que significa que los precios son diferentes para colores o tamaños o ambos.

In [ ]:
create_plots(["Victoria's Secret", "Amazon Victoria's Secret"],swarm = False)

## 3.2 ¿Qué es más caro: los sujetadores de Victoria's Secret o Calvin Klein?

In [ ]:
def category_plots(category, data, y= 'brand',xtitle = 'mrp',money = 'mrp', swarm = True, 
                   figsize1=(15,6), figsize2=(15,6), figsize3=(20,4),
                  filename = None):
    df = data[data['category'].isin(category)]
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=figsize1)
    #ax = sns.boxplot(x=money, hue ='brand', y = 'category',  data=df, ax = axes)
    ax = sns.boxplot(x=money, y= y, data=df, ax=axes, medianprops={"zorder":3})
    if swarm:
        ax = sns.swarmplot(y = y, x=money, data=df, ax = axes, color = 'grey', alpha=0.7)
    ax.xaxis.grid(True)
    ax.set(xlabel=xtitle)
    
    fig, axes = plt.subplots(nrows=1, ncols=1,figsize=figsize2)
    ax = sns.violinplot(y=money, x=y, data=df, ax=axes, cut = 0)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax.yaxis.grid(True)
    ax.set(ylabel=xtitle)
    _ = ax.set_title(", ".join(category))
    if filename!= None:
        fig.savefig(filename + '1.png')
    barplots(money,df, figsize3, x=y, xtitle = xtitle)
    


def barplots(money, df, figsize3, xtitle = 'mrp', x = 'brand', hue = None, palette = light_purple[1:]):
    fig, axes = plt.subplots(nrows=1, ncols=2,figsize=figsize3)
    ax = sns.barplot(y=money, x = x, hue = hue, data = df, palette = palette, ax = axes[0])
    ax.set(ylabel='mean(' + xtitle + ')')
    ax.yaxis.grid(True)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    ax = sns.barplot(y=money, x = x, hue = hue, data = df, palette = palette, estimator=np.median, ax = axes[1])
    ax.set(ylabel='median(' + xtitle + ')')
    display(df.groupby('brand')[money].agg({'median'}).transpose())
    display(df.groupby('brand')[money].agg({'mean'}).transpose())

    ax.yaxis.grid(True)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)


**Pregunta 14**: ¿Qué es más caro: los sujetadores de Victoria's Secret o Calvin Klein?
* Hanky Panky tiene los sujetadores más caros pero tampoco hay tantos (Pregunta 4)
* Victoria's Secret tiene muchos sujetadores que cuestan menos que la mayoría de los sujetadores Calvin's Klein. Sin embargo, VC también tiene sujetadores de mayor precio.
* Los sujetadores Macys Wacoal son más caros que los de otros representantes de Wacoal, ya que venden sujetadores más caros, mientras que otros (amazon y macys) venden solo sujetadores menos costosos.

In [ ]:
for name, group in brands_info.groupby('brand'):
    print(name)
    print((group[(group['category']=='bras')].mrp<40).sum())

In [ ]:
category_plots(['bras'], brands_info,
               figsize1=(15,5), figsize2=(12,4), figsize3=(12,3), filename = 'bras_mrp')

**Pregunta 14**: ¿Qué marca de bragas son las más caras?
* Amazon Hanky Panky tiene las bragas más caras (20 productos). En general, Hanky Panky tiene bragas más caras.
* La mayoría de las bragas de Victoria's Secret tienden a ser un poco más baratas (no olviden que Calvin Klein tiene una oferta de '3 bragas por 33 USD', sin embargo, no se especificó tal oferta para Victoria's Secret, aunque sé que tales ofertas existen en el sitio web) .
* Calvin Klein tiene alrededor de 190 (76% de las bragas de Clavin Klein) con un mrp de 13 USD.
* La mayoría de las bragas Wacoals y Calvin Klein tienen el mismo precio.
* Aunque Topshow tiene una amplia distribución de mrp, tiene bastantes bragas muy baratas.
* La mayoría de las bragas de Calvin Klein de Amazon son más caras que las de la OW.
* Amazon Wacoal tiene distribución bimodal. De hecho, solo vende 2 modelos de bragas. El modelo 'ciao bella tanga panty' tiene un precio de 19 USD, mientras que el modelo 'kiss thong panty' ronda los 11 USD.

In [ ]:
category_plots(['panties'], brands_info, swarm = False, figsize1=(12,5),figsize2=(20,5), figsize3=(15,3))

**Pregunta 15**: ¿Bralettes de qué marca son los más caros?
* Los bralettes de Hanky Panky son los más caros.
* Victoria's Secret es menos costoso que CV.
* Wacoal y Victoria's Secret tienen los bralettes más baratos

In [ ]:
category_plots(['bralettes'], brands_info, figsize1=(20,5),figsize2=(20,5), figsize3=(15,3))

**Question 15**: Bodys of which brand are the most expensive?
* And again Hanky Panky is most expensive one.
* The least expensive are sold by Victoria's Secret and Topshop.

In [ ]:
category_plots(['bodys'], brands_info, figsize1=(15,3), figsize2=(10,3), figsize3=(10,2))

**Pregunta 16**: ¿Ropa deportiva de qué marca es la más cara?
* Macy's Wacoal es, con mucho, la marca más cara en términos de ropa deportiva (principalmente sujetadores deportivos)
* Victoria's Secret y Calvin Klein están en el mismo nivel de precios, sin embargo, la mayoría de los productos son menos costosos en Victoria's Secret
* Topshop es la marca más barata de esta categoría (pero solo tiene 7 productos)

In [ ]:
category_plots(['activewear'], brands_info, figsize1=(15,4), figsize2=(10,3), figsize3=(10,2))

**Pregunta 17**: ¿Prendas de dormir de qué marca son las más caras?
* Hanky Panky tiene los productos más caros
* Victoria's Secret es menos costosa que Calvin Klein.

In [ ]:
category_plots(['sleepwear'], brands_info, figsize1=(15,2), figsize2=(10,3), figsize3=(10,2))

Hanky Panky es la marca más cara en las siguientes categorías: ropa de dormir, bralettes, bragas y sujetadores.
Macy's Wacoal vende la ropa deportiva más cara.

***************************************
**Aprendizajes**:<br>

**Sujetadores**:<br>
Los sujetadores tienen en promedio el mismo precio aproximadamente para otras marcas como Calvin Klein, Victoria's Secret, Topshop y Wacoal. Sin embargo, algunas marcas como Topshop, VS y Macys Wacoal tienen una distribución bimodal (lo que significa que venden ambos: sostenes más baratos y más caros) o varios grandes valores atípicos.

**Bragas**:<br>
La mayoría de las bragas de Wacoal, Topshop, Calvin Klein y Victoria's Secret cuestan entre 10 y 15 USD.
Hanky ​​Panky (OW y Amazon) tiene bragas más caras.
La mayoría de las bragas de Victoria's Secret tienden a ser las más baratas.

**Bralettes**:<br>
Victoria's Secret y Wacoal tienen los bralettes más baratos.

**Cuerpos**:<br>
Solo 5 marcas venden carrocerías. Los menos costosos los venden Victoria's Secret y Topshop.

**Ropa deportiva**:<br>
Victoria's Secret y Calvin Klein están en el mismo nivel de precios, sin embargo, la mayoría de los productos son menos costosos en Victoria's Secret.
***************************************

**Pregunta 18:** ¿Victoria's Secret o Calvin Klein son más baratos?
Al principio vimos que Calvin Klein era un poco más caro que Victoria's Secret pero ¿está en todas las categorías?

La ropa de dormir, la ropa deportiva, las bragas y los barlettes de Calvin Klein son más caros que los de Victoria's Secret.

In [ ]:
def barplots_extra(money, df, figsize3, xtitle = 'mrp', x = 'brand', hue = None, palette = light_purple[1:]):
    fig, axes = plt.subplots(nrows=2, ncols=1,figsize=figsize3)
    plt.subplots_adjust(hspace = 0.5, wspace=0.5)
    ax = sns.barplot(y=money, x = x, hue = hue, data = df, palette = palette, ax = axes[0])
    ax.set_ylabel('mean(' + xtitle + ')', fontsize=13)
    ax.set_xlabel('category', fontsize=12)
    ax.yaxis.grid(True)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    _= ax.tick_params(axis='both', which='major', labelsize=14)

    ax = sns.barplot(y=money, x = x, hue = hue, data = df, palette = palette, estimator=np.median, ax = axes[1])
    ax.set_ylabel('median(' + xtitle + ')', fontsize=13)
    display(df.groupby('brand')[money].agg({'median'}).transpose())
    ax.set_xlabel('category', fontsize=12)
    _ = ax.set_xticklabels(ax.get_xticklabels(), rotation=90)
    _= ax.tick_params(axis='both', which='major', labelsize=14)
    ax.yaxis.grid(True)


In [ ]:
sns.set(font_scale = 1.2)
sns.set_style('white', {'legend.frameon':True})
df = brands_info[brands_info['brand'].isin(['Calvin Klein', "Victoria's Secret"])]
df = df[df['product_category']!='Other']
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(15,6))
ax = sns.boxplot(y = 'category', x="mrp", hue='brand', data=df, ax=axes, palette = [light_purple[2], light_purple[5]])
#ax = sns.swarmplot(y = 'category', hue = 'brand', x="mrp", data=df, ax = axes, color = 'grey')
_= ax.tick_params(axis='both', which='major', labelsize=13)
ax.set_xlabel('mrp', fontsize=13)
ax.set_ylabel('category', fontsize=13)
ax.xaxis.grid(True)
fig.savefig('calvin_victoria_boxplot.png')

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(15,6))
ax = sns.violinplot(y="mrp", hue ='brand', x = 'category',cut = 0, split=True, data=df, ax = axes, palette = [light_purple[2], light_purple[5]])
ax.yaxis.grid(True)
_= ax.tick_params(axis='both', which='major', labelsize=13)
ax.set_ylabel('mrp', fontsize=13)
ax.set_xlabel('category', fontsize=13)
fig.savefig('calvin_victoria_violin.png')

barplots_extra('mrp', df, (6,10), hue = 'brand', x='category', palette = [light_purple[2], light_purple[5]])


In [ ]:
def find_discount(data):
    data['diff'] = (data['mrp'] - data['price']).round(2)
    data['diff_categories'] = ''
    data.loc[(data['diff'] < 9) & (data['diff'] != 0.0), 'diff_categories'] = 'low'
    data.loc[(data['diff'] >= 9) & (data['diff'] < 18), 'diff_categories'] = 'middle'
    data.loc[(data['diff'] >= 18), 'diff_categories'] = 'high'
    data.loc[(data['diff'] == 0.0), 'diff_categories'] = 'no'   
    data['discount'] = "not discounted"
    data.loc[(data['diff'] > 0.0), 'discount'] = 'discounted'   
    return data

In [ ]:
brands_info = find_discount(brands_info)

In [ ]:
def stacked_all(data, category, name = None, discount_cat = 'diff_categories'):
    fig, ax = plt.subplots(nrows=2, ncols=1,figsize=(15,10))
    tab = pd.crosstab(data[discount_cat], data[category]).transpose()
    if 'high (>=50%)' in tab.columns:
        ordering = ['high (>=50%)','middle (20%>=x<50%)','low (<20%)','no']
    else:
        ordering = ['high','middle','low','no']
    cor = [x for x in ordering if x in tab.columns] + [x for x in tab.columns if x not in ordering]
    tab = tab[cor]

    tab_prop = tab.div(tab.sum(1).astype(float), axis=0)  
    tab_pro = tab_prop.append(pd.DataFrame([1])).drop(0, axis=1) 
    display(tab_pro.transpose())
    ##f7caf2
    colors = ["#755791","#c06c84", "#f67280", "#f8b195"] 
    

    tab_pro.reindex(tab_pro.index.tolist()).plot(kind="bar", stacked=True, ax = ax[0], fontsize = 9, color = colors)  
    ax[0].yaxis.grid(True)
    df = tab.transpose()
    bbox=[0, 0, 1, 1]
    ax[1].axis('off')
    mpl_table = ax[1].table(cellText = df.values, rowLabels = df.index, 
                            colLabels=df.columns, bbox = bbox)

    plt.legend(fontsize=9)
    #ax[0].set_xlabel("Difference between RMP and price", fontsize = 10 )
    ax[0].set_title(name)


In [ ]:
def stacked_all(data, category, name = None):
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(6,3))
    tab = pd.crosstab(data['diff_categories'], data[category]).transpose()
    print(name)
    display(tab)

    tab_prop = tab.div(tab.sum(1).astype(float), axis=0)  
    tab_pro = tab_prop.append(pd.DataFrame([1])).drop(0, axis=1) 
    display(tab_pro.transpose())

    tab_pro.reindex(tab_pro.index.tolist()).plot(kind="bar", stacked=True, ax = ax, fontsize = 9)


**Question 19**: Which product category falls into which discount category?
* Around 56% of bras and activewear and 52% of bralettes are discounted. 
* Only 20% of panties are discounted (do not forget there are special offers like 3 for 33 USD etc)
* 25% of babydolls and 34% of bras have a high discount. 

In [ ]:
stacked_all(brands_info, 'category')

In [ ]:
tab = pd.crosstab(brands_info['discount'], brands_info['brand']).transpose()
display(tab.transpose())

tab_prop = tab.div(tab.sum(1).astype(float), axis=0)  
tab_pro = tab_prop.append(pd.DataFrame([1])).drop(0,axis=1) 
tab_pro.drop(tab_pro.index[12], inplace = True)
display(tab_pro.transpose())
colors = ["#755791", "#f67280"]
fig, ax = plt.subplots(nrows=1, ncols=1,figsize=(8,6))
_ = tab_pro.reindex(tab_pro.index.tolist()).plot(kind="bar", stacked=True, ax = ax, fontsize = 12, color = colors)
_= ax.tick_params(axis='both', which='major', labelsize=14)
plt.tight_layout()
fig.savefig('percent_discount.png')


In [ ]:
def stacked_all_brands(data, discount_cat = 'diff_categories', figsize = (15,5)):
    fig, ax = plt.subplots(nrows=1, ncols=1,figsize=figsize)
    tab = pd.crosstab(data[discount_cat], data['brand']).transpose()
    if 'high (>=50%)' in tab.columns:
        ordering = ['high (>=50%)','middle (20%>=x<50%)','low (<20%)','no']
    else:
        ordering = ['high','middle','low','no']
    cor = [x for x in ordering if x in tab.columns] + [x for x in tab.columns if x not in ordering]
    tab = tab[cor]
    display(tab.transpose())

    tab_prop = tab.div(tab.sum(1).astype(float), axis=0)  
    tab_pro = tab_prop.append(pd.DataFrame([1])).drop(0, axis=1) 
    display(tab_pro.transpose())
    ##f7caf2
    colors = ["#755791","#c06c84", "#f67280", "#f8b195"] 
    tab_pro.reindex(tab_pro.index.tolist()).plot(kind="bar", stacked=True, ax = ax, fontsize = 9, color = colors)
    ax.yaxis.grid(True)

## 3.2 ¿Quién ofrece mayor descuento?
**Pregunta 21**: ¿Qué marcas ofrecen mayores descuentos?
Tomaremos en consideración solo las marcas con una gran cantidad de productos con descuento.

* Victoria's Secret, Macys Calvin Klein y Topshop tienen varios descuentos realmente grandes (más de 40 USD).
* Hanky Panky (OW y Macys) parece tener el mayor descuento (promedio y mediana) pero solo porque hay una cantidad insignificante de artículos con descuento en total.

In [ ]:
df = brands_info[brands_info['diff']>0]
category_plots(brands_info['category'].unique(), df, swarm = True, money = 'diff', xtitle = 'difference')

**Pregunta 22**: ¿Cuáles son los productos con descuento superior a 40 USD?
Como era de esperar, son artículos un poco más caros como sostenes, barlettes, babydools y un mameluco.

In [ ]:
brands_info[brands_info['diff']>40][['product_name','brand','category','diff', 'price', 'mrp', 'color']]

Algunos de los artículos más rebajados:

In [ ]:
html = "<table><tr><td>Victorias Secret: bra<img src='https://images-na.ssl-images-amazon.com/images/I/713DKKJk4dL._UY445_.jpg' style='width: 200px;'></td><td>Victorias Secret: slip<img src='https://images-na.ssl-images-amazon.com/images/I/61CWtyQDgjL._UL1500_.jpg' style='width: 200px;'></td><td>Topshop: bra by bluebella<img src='http://media.topshop.com/wcsstore/TopShopDE/images/catalog/TS43C62LBLK_Zoom_F_1.jpg' style='width: 200px;'></td></tr></table>"
display(HTML(html))

**Pregunta 23:** ¿Cuál es el artículo con menos descuento?
Así es la braguita de Amazon Calvin Klein que ha sido rebajada solo por 1,25 USD.

In [ ]:
html = "<table><tr><td>Amazon Calvin Klein: least discounted panty<img src='https://images-na.ssl-images-amazon.com/images/I/81AmIezUmWL._UL1500_.jpg' style='width: 200px;'></td></tr></table>"
display(HTML(html))

In [ ]:
min_diff = np.min(brands_info[brands_info['diff']>0]['diff'])
brands_info[brands_info['diff']==min_diff]

## 3.3 Ratio de RMP al precio. Valor de descuento real.

Large discounts are good but no panties would get into high level discount unless they cost around 40 euros. As we have seen the average price of the panties is around 15 dollars. It will be more informative to look at the percentage of the offered discount to the price. So, if the rmp is 14 dollars and price is 7 dollars. There is a 50% discount.

In [ ]:
def find_percentage_discount(data):
    data['diff_per'] = (data['diff'] * 100  / data['mrp']).round(2)  

    data['diff_per_categories'] = ''
    data.loc[(data['diff_per'] < 20) & (data['diff_per'] != 0.0), 'diff_per_categories'] = 'low (<20%)'
    data.loc[(data['diff_per'] >= 20) & (data['diff_per'] < 50), 'diff_per_categories'] = 'middle (20%>=x<50%)'
    data.loc[(data['diff_per'] >= 50), 'diff_per_categories'] = 'high (>=50%)'
    data.loc[(data['diff_per'] == 0.0), 'diff_per_categories'] = 'no'   
    return data

In [ ]:
brands_info = find_percentage_discount(brands_info)

## 3.3.1 ¿Qué marca ofrece qué porcentaje de descuento?

**Pregunta 24.1:** ¡Ventas! Hasta 50%. ¿Qué porcentaje de descuento ofrece cada marca?
* ¡La mayoría de los descuentos de Victoria's Secret rondan el 50 %, con un valor atípico de >=80 % de descuento!
* Topshop tiene una distribución bimodal con descuentos en torno al 40% y 60-70% pero con una mediana del 50%.
* Calvin Klein tiene la mayoría de los descuentos alrededor del 46 % y Amazon Calvin Klein alrededor del 36 %, pero Amazon tiene más artículos con >=50 % de descuento.
* Wacoal ofrece 30% o 40% de descuento y Macy's Wacoal también tiene una distribución bimodal con alrededor de 30% y 45% de descuento.
* Hanky Panky no ofrece gran parte del descuento.

In [ ]:
df = brands_info
df = df[df['diff_per']>0]
category_plots(df['category'].unique(), df, swarm = True, money = 'diff_per', xtitle='% discount',
               figsize1=(15,5), figsize2=(15,4), figsize3=(10,2))

In [ ]:
df[df['diff_per']>50].groupby('brand').size()

In [ ]:
df = brands_info[brands_info['diff_per']>=75]
df.drop_duplicates(['product_name','price','mrp','description'])[['product_name','price','mrp', 'diff_per', 'color', 'brand']]

**Pregunta 24.2:** ¡Ventas! Hasta 50%. ¿Qué porcentajes de descuento son comunes para cada categoría?<br>
Como la mayoría de los artículos provienen de Victoria's Secret, esos gráficos están fuertemente influenciados por esta marca.

* Bragas, bralettes, sujetadores, ropa deportiva tienen casi el mismo descuento promedio (alrededor del 50%). Sin embargo, más bragas tienen un descuento >=60% que cualquier otra categoría.
* Algunos bañadores tienen más descuento (más del 55%) que otros (menos del 50%).
* Bodys, babydolls, Otros y pack tienen un descuento medio en torno al 40%.

In [ ]:
df = brands_info
df = df[df['diff_per']>0]
category_plots(df['category'].unique(), df, y='category',swarm = True, money = 'diff_per', xtitle='% discount',
               figsize1=(12,5), figsize2=(12,4), figsize3=(10,2))

In [ ]:
df[df['diff_per']>60].groupby('category').size()

In [ ]:
#**Question 25:** Sales! Up to 50%. What percentage disconts are offered by each brand for each category?
#* Around 20% (709) of panties have a discount >=20% (with 10% (348) of panties having >=50% discount)
#* 15% (190) of bras, 24% (193) of bralettes, 14% (32) of swimsuits and 12% (78) of activewear have a >=50% discount.
#stacked_all(brands_info, 'category', name = 'All Brands', discount_cat='diff_per_categories')

**Pregunta 26:** ¿Qué marca ofrece un mayor porcentaje de descuento en cada categoría?

## bragas
* _más del 50% de descuento_: 44% (22) de Amazon Calvin Klein, 17% (37) de Topshop y 16% (278) de Victoria's Secret
* _más del 20% de descuento_: 76% (38) de Amazon Calvin Klein, 35% (54) de Topshop y 18,8% (310) de Victoria's Secret, 18% (46) de Calvin Klein, 16% (49) de Macy Calvin Klein


* Victoria's Secret ofrece el mayor descuento promedio en bragas.


Aprendizajes:
* Si eres fanático de Calvin Klein, primero revisaría Amazon Calvin Klein, ya que es posible que tengan más artículos con grandes descuentos, pero OW tiene más bragas en general. Y no te olvides de los packs (pero Calvin Klein también tenía una oferta "3 por 33").
* Si te gusta Wacoal, consulta primero OW porque la mayoría de las bragas tendrán un descuento de alrededor del 40 %, pero Macys podría tener más productos con un descuento de >45 %.
* Si quieres buscar los descuentos, ve a Victoria's Secret por bragas.

In [ ]:
def perform_viz(category, figsize1=(12,4), figsize2=(12,4), figsize3=(10,3)):
    df = brands_info
    df1 = df[df['category']==category]
    stacked_all_brands(df1, discount_cat='diff_per_categories')
    df1 = df[df['diff_per']>0]
    category_plots([category], df1, swarm = True, money = 'diff_per', xtitle = '% discount',
               figsize1=figsize1, figsize2=figsize2, figsize3=figsize3)
    
perform_viz('panties')

## Bras
* _more than 50% discount_: 16% (4) of Amazon Calvin Klein, 17% (22) of Topshop and 33% (156) of Victoria's Secret 
* _more than 20% discount_: 48% (16) of Amazon Calvin Klein, 36% (47) of Topshop and 85% (405) of Victoria's Secret, 50% (46) of Calvin Klein, 17% (28) of Macy Calvin Klein, 30% (67) of Macys Wacoal, 50% (41) of Wacoal <br>
<br>
* Victoria's Secret offers the largest relative number of discounted bras.
* Most bras of Victoria's Secret have a discount of 49%
* Topshop and Victoria's Secret offer more products with a discount over 50% than other brands.
* Macys Calvin Klein has a discount of around 47% on most of their discounted bras. Calvin Klein has more discounted bra but with around 42% discount, however, it still has outliers.
* Wacoal has again a bimodal discount distribution around 30% and 40%


Learnings: 
* Check Topshop and Victoria's Secret for (highly) discounted bras.
* If you like Wacoal, check first Macys as the majority of items have a discount of 46%, comparing to Wacoal's 40% (as well as the average discount is around 42% at Macys in comparison to OW with 37%)
* For Calvin Klein check both websites.

In [ ]:
perform_viz('bras',figsize1=(12,4), figsize2=(20,3), figsize3=(10,5))

## Bralettes

In [ ]:
perform_viz('bralettes', figsize1=(12,4), figsize2=(20,3), figsize3=(10,3))

In [ ]:
perform_viz('activewear', figsize1=(12,4), figsize2=(20,3), figsize3=(10,3))

## Paquetes
* La mayoría de los paquetes de Amazon Calvin Klein tienen descuento.

In [ ]:
perform_viz('pack', figsize1=(12,4), figsize2=(20,3), figsize3=(10,3))

## ropa de dormir

In [ ]:
perform_viz('sleepwear', figsize1=(12,4), figsize2=(20,3), figsize3=(10,3))

**Otras categorías**

In [ ]:
cats = df['category'].unique()
cats = set(cats)- set(['panties', 'bras', 'activewear', 'bralettes', 'pack', 'sleepwear'])
#df1 = brands_info[brands_info['brand'].isin(brands)]
df1 = df[df['category'].isin(cats)]
stacked_all_brands(df1, discount_cat='diff_per_categories')

df1 = df[df['diff_per']>0]
category_plots(cats, df1, swarm = True, money = 'diff_per', xtitle = '% discount',
              figsize1=(12,4), figsize2=(20,3), figsize3=(10,3))

### Comparación de precios de Calvin Klein vs Victoria's Secret

In [ ]:
df = brands_info[brands_info['brand'].isin(['Calvin Klein', "Victoria's Secret"])]
df = df[df['product_category']!='Other']
fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(15,6))
ax = sns.boxplot(y = 'category', x="price", hue='brand', data=df, ax=axes, palette = [light_purple[2], light_purple[5]])
#ax = sns.swarmplot(y = 'category', hue = 'brand', x="mrp", data=df, ax = axes, color = 'grey')
_= ax.tick_params(axis='both', which='major', labelsize=13)
ax.set_xlabel('price', fontsize=13)
ax.set_ylabel('category', fontsize=13)
ax.xaxis.grid(True)
fig.savefig('calvin_victoria_boxplot.png')

fig, axes = plt.subplots(nrows=1, ncols=1,figsize=(15,6))
ax = sns.violinplot(y="price", hue ='brand', x = 'category',cut = 0, split=True, data=df, ax = axes, palette = [light_purple[2], light_purple[5]])
ax.yaxis.grid(True)
_= ax.tick_params(axis='both', which='major', labelsize=13)
ax.set_ylabel('price', fontsize=13)
ax.set_xlabel('category', fontsize=13)
fig.savefig('calvin_victoria_violin.png')

barplots_extra('price', df, (6,10), hue = 'brand', x='category', palette = [light_purple[2], light_purple[5]])

In [ ]:
#brands_info.to_csv('data/final_data.csv')